
# Reading Table from RDBMS [SQL Server]
> *using jdbc connector to tap into a relational database system* 


<br>
  
<img width="900px" src ='https://owshqblobstg.blob.core.windows.net/stgfiles/png_files/rdbms_bronze0.png'>
  
<br>

In [0]:
# all these configurations can be saved into the key vault [azure or databricks]
# meet the data source = sql server
# specifying connection string
# add info related to connectivity
jdbcHostname = "159.203.144.206"
jdbcDatabase = "owshq"
jdbcUsername = "sa"
jdbcPassword = "IlS27OpKxw9EYObU80dz"
jdbcPort = 1433
jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2};user={3};password={4};".format(jdbcHostname, jdbcPort, jdbcDatabase, jdbcUsername, jdbcPassword)

In [0]:
# building the connection string to access resource
# if not installed, configure sql server driver
# com.microsoft.sqlserver.jdbc.SQLServerDriver
jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2}".format(jdbcHostname, jdbcPort, jdbcDatabase)
connectionProperties = {
  "user" : jdbcUsername,
  "password" : jdbcPassword,
  "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

In [0]:
# set query to be issued against sql server
pushdown_query = "(SELECT DISTINCT user_id, [plan], status, payment_method, subscription_term, payment_term FROM subscription WHERE status = 'Active') active_subscription"

# use dataframe to read from jdbc
# url = constructured above
# table = the query build 
# properties = connection
# load data into dataframe
df_active_subscription = spark.read.jdbc(url=jdbcUrl, table=pushdown_query, properties=connectionProperties)

# show dataframe data
display(df_active_subscription)

user_id,plan,status,payment_method,subscription_term,payment_term
312,Platinum,Active,WeChat Pay,Daily,Monthly
433,Platinum,Active,Cheque,Daily,Monthly
1074,Basic,Active,Alipay,Triennal,Annual
1516,Silver,Active,Cash,Biennal,Payment in advance
2194,Professional,Active,Google Pay,Annual,Full subscription
2214,Standard,Active,WeChat Pay,Monthly,Payment in advance
2313,Free Trial,Active,Alipay,Triennal,Full subscription
2317,Silver,Active,Credit card,Biennal,Full subscription
2893,Professional,Active,WeChat Pay,Annual,Payment in advance
3996,Diamond,Active,Visa checkout,Quinquennal,Annual


In [0]:
print(df_active_subscription.rdd.getNumPartitions())

1

In [0]:
df_active_subscription.count()

Out[4]: 15


>* generate new files to relational database sql server  
python3.9 cli.py 'mssql'

In [0]:
# read table subscription from sql server
# using additional configurations to read at scale
# register as a spark sql object
df_subscription = spark.read.jdbc(url=jdbcUrl, table="subscription", column="user_id", lowerBound=1, upperBound=100000, numPartitions=100, properties=connectionProperties)
df_subscription.registerTempTable("vw_subscription")
display(df_subscription)

/databricks/spark/python/pyspark/sql/dataframe.py:146: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
 warnings.warn(

id,uid,plan,status,payment_method,subscription_term,payment_term,user_id,dt_current_timestamp
7549,c51d4631-dbf3-422c-8b78-8fcdb2aa44e2,Platinum,Active,Cheque,Daily,Monthly,433,2022-08-15T17:33:36.500+0000
7275,79ff49f2-55c5-4fe6-92ea-c64bb96c6afd,Silver,Blocked,Apple Pay,Triennal,Full subscription,146,2022-08-15T17:33:36.500+0000
3802,7954062e-6581-4443-a5fb-bf63c2858bd7,Starter,Blocked,Alipay,Triennal,Payment in advance,719,2022-08-15T17:33:36.500+0000
7179,1c7aba55-475a-496a-a915-49ff84b41c99,Gold,Pending,Cheque,Triennal,Annual,241,2022-08-15T17:33:36.500+0000
266,c68a51e7-c4a6-4206-bded-fd5d6d3a9795,Silver,Idle,Bitcoins,Daily,Payment in advance,118,2022-08-15T17:33:36.500+0000
4273,a6c3bfe6-652f-4f32-9025-5c5603284bc6,Platinum,Active,WeChat Pay,Daily,Monthly,312,2022-08-15T17:33:36.500+0000
3320,95f6ce41-b552-4acf-9c9e-8598864d4737,Silver,Pending,WeChat Pay,Quinquennal,Full subscription,1600,2022-08-15T17:33:36.500+0000
400,c0460f39-217b-470b-87bd-97f962c2c618,Standard,Idle,Apple Pay,Daily,Annual,1594,2022-08-15T17:33:36.500+0000
9492,8592267d-bb65-418a-a1b4-98e9eba85d82,Gold,Idle,Debit card,Lifetime,Monthly,1736,2022-08-15T17:33:36.500+0000
3571,d8ae36d0-7430-4dc4-99c0-2e0bb1cb7ab3,Basic,Idle,Apple Pay,Quinquennal,Full subscription,1935,2022-08-15T17:33:36.500+0000


In [0]:
print(df_subscription.rdd.getNumPartitions())

100

In [0]:
%sql

-- count amount of rows
-- 5.020
-- 52.220
-- 55.620
-- 100
SELECT COUNT(*) 
FROM vw_subscription

count(1)
100


In [0]:
%sql

-- query data
SELECT *
FROM vw_subscription

id,uid,plan,status,payment_method,subscription_term,payment_term,user_id,dt_current_timestamp
7549,c51d4631-dbf3-422c-8b78-8fcdb2aa44e2,Platinum,Active,Cheque,Daily,Monthly,433,2022-08-15T17:33:36.500+0000
7275,79ff49f2-55c5-4fe6-92ea-c64bb96c6afd,Silver,Blocked,Apple Pay,Triennal,Full subscription,146,2022-08-15T17:33:36.500+0000
3802,7954062e-6581-4443-a5fb-bf63c2858bd7,Starter,Blocked,Alipay,Triennal,Payment in advance,719,2022-08-15T17:33:36.500+0000
7179,1c7aba55-475a-496a-a915-49ff84b41c99,Gold,Pending,Cheque,Triennal,Annual,241,2022-08-15T17:33:36.500+0000
266,c68a51e7-c4a6-4206-bded-fd5d6d3a9795,Silver,Idle,Bitcoins,Daily,Payment in advance,118,2022-08-15T17:33:36.500+0000
4273,a6c3bfe6-652f-4f32-9025-5c5603284bc6,Platinum,Active,WeChat Pay,Daily,Monthly,312,2022-08-15T17:33:36.500+0000
3320,95f6ce41-b552-4acf-9c9e-8598864d4737,Silver,Pending,WeChat Pay,Quinquennal,Full subscription,1600,2022-08-15T17:33:36.500+0000
400,c0460f39-217b-470b-87bd-97f962c2c618,Standard,Idle,Apple Pay,Daily,Annual,1594,2022-08-15T17:33:36.500+0000
9492,8592267d-bb65-418a-a1b4-98e9eba85d82,Gold,Idle,Debit card,Lifetime,Monthly,1736,2022-08-15T17:33:36.500+0000
3571,d8ae36d0-7430-4dc4-99c0-2e0bb1cb7ab3,Basic,Idle,Apple Pay,Quinquennal,Full subscription,1935,2022-08-15T17:33:36.500+0000



<br>
  
<img width="400px" src ='https://owshqblobstg.blob.core.windows.net/stgfiles/png_files/dl_pq.png'>
  
<br>

In [0]:
# write into parquet
# processing zone
# using overwrite 
df_subscription.write.mode("overwrite").parquet("/mnt/processing/parquet/batch/subscription")


<br>
  
<img width="900px" src ='https://owshqblobstg.blob.core.windows.net/stgfiles/png_files/dl_delta_bronze0.png'>
  
<br>


In [0]:
# write into delta
# delta architecture zone
# using overwrite
df_subscription.write.mode("overwrite").format("delta").save("dbfs:/mnt/owshq/delta/batch/bronze/subscription")